<a href="https://colab.research.google.com/github/EderBraz/Coursera_Capstone/blob/master/Toronto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Part 1

###Importing libraries

In [0]:
import pandas as pd
import numpy as np
import folium
from sklearn.cluster import KMeans

### Scraping the table and creating a dataframe

In [0]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = pd.read_html(url)
df = page[0]
df.head(10)

###Cleaning data and pre-processing

In [56]:
df1 = df[df.Borough != 'Not assigned']
df1 = df1.sort_values(by=['Postal code','Borough'])

df1.reset_index(inplace=True)
df1.drop('index',axis=1,inplace=True)
df1['Neighborhood'] = (df1['Neighborhood'].replace('/',',', regex=True))
df1.head()


,Postal code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern , Rouge"
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek"
2,M1E,Scarborough,"Guildwood , Morningside , West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [55]:
df1.shape

(103, 3)

##Part 2

###Extracting csv with coords and creating the dataframe

In [52]:
link = "http://cocl.us/Geospatial_data"
df2 = pd.read_csv(link)

df2.head()
df2.shape

(103, 3)

###Merging dataframes

In [0]:
df2.columns = ["Postal code", 'Latitude', 'Longitude']
cols = df2.columns.to_list()

In [60]:
df_new = pd.merge(df1, df2, on='Postal code')
df_new.head()

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern , Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood , Morningside , West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


##Part 3

###Total Boroughs

In [65]:
df_new['Borough'].nunique()

10

In [77]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df_new['Borough'].unique()),
        df_new.shape[0]))

The dataframe has 10 boroughs and 103 neighborhoods.


###Neighborhoods

In [66]:
df_new['Borough'].value_counts()


North York          24
Downtown Toronto    19
Scarborough         17
Etobicoke           12
Central Toronto      9
West Toronto         6
East Toronto         5
York                 5
East York            5
Mississauga          1
Name: Borough, dtype: int64

In [82]:
lat, long = 43.653963, -79.387207
map_toronto = folium.Map(location=[lat, long], zoom_start=10)
# add markers to map
for lat, long, borough, neighborhood in zip(df_new['Latitude'], df_new['Longitude'], df_new['Borough'], df_new['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, long],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto